In [1]:
import pandas as pd

In [2]:
p0_names = [
    'Nessus SYN Scanner',
    'Nessus Scan Information',
    'OS Identification',
]

## Cleaning OpenVAS

In [3]:
vas = pd.read_excel('openvas.xlsx')

In [4]:
vas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Plugin ID                 173 non-null    int64         
 1   CVE                       36 non-null     object        
 2   CVSS v2.0 Base Score      57 non-null     float64       
 3   Risk                      56 non-null     object        
 4   Host                      173 non-null    object        
 5   Protocol                  173 non-null    object        
 6   Port                      173 non-null    int64         
 7   Name                      173 non-null    object        
 8   Synopsis                  173 non-null    object        
 9   Description               173 non-null    object        
 10  Solution                  97 non-null     object        
 11  See Also                  75 non-null     object        
 12  Plugin Output         

In [5]:
vas = vas.drop_duplicates(keep='first')
vas

,Plugin ID,CVE,CVSS v2.0 Base Score,Risk,Host,Protocol,Port,Name,Synopsis,Description,...,See Also,Plugin Output,BID,XREF,MSKB,Plugin Publication Date,Plugin Modification Date,Metasploit,Core Impact,CANVAS
0,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,http://tomcat.apache.org/connectors-doc/\nhttp...,\nThe connector listing on this port supports ...,NaN,NaN,NaN,2006-04-05,2019-11-22,NaN,NaN,NaN
1,18261,NaN,NaN,NaN,192.168.1.172,tcp,0,Apache Banner Linux Distribution Disclosure,The name of the Linux distribution running on ...,Nessus was able to extract the banner of the A...,...,NaN,\nThe Linux distribution detected was : \n - U...,NaN,NaN,NaN,2005-05-15,2022-03-21,NaN,NaN,NaN
2,48204,NaN,NaN,NaN,192.168.1.172,tcp,80,Apache HTTP Server Version,It is possible to obtain the version number of...,The remote host is running the Apache HTTP Ser...,...,https://httpd.apache.org/,\n URL : http://192.168.1.172/\n Vers...,NaN,IAVT:0001-T-0030;IAVT:0001-T-0530,NaN,2010-07-30,2023-08-17,NaN,NaN,NaN
3,134862,CVE-2020-1745,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
4,134862,CVE-2020-1938,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,52703,NaN,NaN,NaN,192.168.1.172,tcp,21,vsftpd Detection,An FTP server is listening on the remote port.,"The remote host is running vsftpd, an FTP serv...",...,http://vsftpd.beasts.org/,\n Source : 220 (vsFTPd 2.3.4)\n Version : ...,NaN,NaN,NaN,2011-03-17,2019-11-22,NaN,NaN,NaN
169,11424,NaN,NaN,NaN,192.168.1.172,tcp,80,WebDAV Detection,The remote server is running with WebDAV enabled.,WebDAV is an industry standard extension to th...,...,NaN,NaN,NaN,NaN,NaN,2003-03-20,2011-03-14,NaN,NaN,NaN
170,10150,NaN,NaN,NaN,192.168.1.172,udp,137,Windows NetBIOS / SMB Remote Host Information ...,It was possible to obtain the network name of ...,The remote host is listening on UDP port 137 o...,...,NaN,The following 7 NetBIOS names have been gather...,NaN,NaN,NaN,1999-10-12,2021-02-10,NaN,NaN,NaN
171,135860,NaN,NaN,NaN,192.168.1.172,tcp,445,WMI Not Available,WMI queries could not be made against the remo...,WMI (Windows Management Instrumentation) is no...,...,https://docs.microsoft.com/en-us/windows/win32...,Can't connect to the 'root\CIMV2' WMI namespace.,NaN,NaN,NaN,2020-04-21,2024-08-14,NaN,NaN,NaN


In [6]:
def rm_by_col(df, columns):
    df_non_nan = df.dropna(subset=columns)
    df_nan = df[df[columns[0]].isna() | df[columns[1]].isna()]

    # Drop duplicates in the non-NaN part while keeping the first occurrence based on the specified columns
    df_non_nan_no_duplicates = df_non_nan.drop_duplicates(subset=columns, keep='first')

    # Concatenate the non-NaN part with removed duplicates and the original NaN part
    df_result = pd.concat([df_non_nan_no_duplicates, df_nan], ignore_index=True)
    return df_result


In [7]:
vas = vas.drop_duplicates(subset=['CVE', 'Description', 'Port'], keep='first')
vas

,Plugin ID,CVE,CVSS v2.0 Base Score,Risk,Host,Protocol,Port,Name,Synopsis,Description,...,See Also,Plugin Output,BID,XREF,MSKB,Plugin Publication Date,Plugin Modification Date,Metasploit,Core Impact,CANVAS
0,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,http://tomcat.apache.org/connectors-doc/\nhttp...,\nThe connector listing on this port supports ...,NaN,NaN,NaN,2006-04-05,2019-11-22,NaN,NaN,NaN
1,18261,NaN,NaN,NaN,192.168.1.172,tcp,0,Apache Banner Linux Distribution Disclosure,The name of the Linux distribution running on ...,Nessus was able to extract the banner of the A...,...,NaN,\nThe Linux distribution detected was : \n - U...,NaN,NaN,NaN,2005-05-15,2022-03-21,NaN,NaN,NaN
2,48204,NaN,NaN,NaN,192.168.1.172,tcp,80,Apache HTTP Server Version,It is possible to obtain the version number of...,The remote host is running the Apache HTTP Ser...,...,https://httpd.apache.org/,\n URL : http://192.168.1.172/\n Vers...,NaN,IAVT:0001-T-0030;IAVT:0001-T-0530,NaN,2010-07-30,2023-08-17,NaN,NaN,NaN
3,134862,CVE-2020-1745,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
4,134862,CVE-2020-1938,7.5,Critical,192.168.1.172,tcp,8009,Apache Tomcat AJP Connector Request Injection ...,There is a vulnerable AJP connector listening ...,A file read/inclusion vulnerability was found ...,...,http://www.nessus.org/u?8ebe6246\nhttp://www.n...,\nNessus was able to exploit the issue using t...,NaN,CISA-KNOWN-EXPLOITED:2022/03/17;CEA-ID:CEA-202...,NaN,2020-03-24,2024-07-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,52703,NaN,NaN,NaN,192.168.1.172,tcp,21,vsftpd Detection,An FTP server is listening on the remote port.,"The remote host is running vsftpd, an FTP serv...",...,http://vsftpd.beasts.org/,\n Source : 220 (vsFTPd 2.3.4)\n Version : ...,NaN,NaN,NaN,2011-03-17,2019-11-22,NaN,NaN,NaN
169,11424,NaN,NaN,NaN,192.168.1.172,tcp,80,WebDAV Detection,The remote server is running with WebDAV enabled.,WebDAV is an industry standard extension to th...,...,NaN,NaN,NaN,NaN,NaN,2003-03-20,2011-03-14,NaN,NaN,NaN
170,10150,NaN,NaN,NaN,192.168.1.172,udp,137,Windows NetBIOS / SMB Remote Host Information ...,It was possible to obtain the network name of ...,The remote host is listening on UDP port 137 o...,...,NaN,The following 7 NetBIOS names have been gather...,NaN,NaN,NaN,1999-10-12,2021-02-10,NaN,NaN,NaN
171,135860,NaN,NaN,NaN,192.168.1.172,tcp,445,WMI Not Available,WMI queries could not be made against the remo...,WMI (Windows Management Instrumentation) is no...,...,https://docs.microsoft.com/en-us/windows/win32...,Can't connect to the 'root\CIMV2' WMI namespace.,NaN,NaN,NaN,2020-04-21,2024-08-14,NaN,NaN,NaN


In [8]:
vas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 169 entries, 0 to 172
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Plugin ID                 169 non-null    int64         
 1   CVE                       36 non-null     object        
 2   CVSS v2.0 Base Score      57 non-null     float64       
 3   Risk                      56 non-null     object        
 4   Host                      169 non-null    object        
 5   Protocol                  169 non-null    object        
 6   Port                      169 non-null    int64         
 7   Name                      169 non-null    object        
 8   Synopsis                  169 non-null    object        
 9   Description               169 non-null    object        
 10  Solution                  96 non-null     object        
 11  See Also                  73 non-null     object        
 12  Plugin Output             1

In [9]:
vas.to_excel('new_openvas.xlsx')

## Cleaning Nessus

In [10]:
nes = pd.read_excel('nessus.xlsx')

In [11]:
nes = nes[nes['IP'] == '192.168.1.172']

## Merging the DataFrames

In [12]:
nes.rename(columns={'CVEs': 'CVE'}, inplace=True)

# Step 2: Merge the DataFrames on 'CVE'
merged_df = pd.merge(vas, nes, on='CVE', how='inner')

# Display the resulting DataFrame
merged_df.drop_duplicates(inplace=True)
merged_df

,Plugin ID,CVE,CVSS v2.0 Base Score,Risk,Host,Protocol,Port_x,Name,Synopsis,Description,...,Result ID,Impact,Solution_y,Affected Software/OS,Vulnerability Insight,Vulnerability Detection Method,Product Detection Result,BIDs,CERTs,Other References
0,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,7155caa0-53c3-4379-8fa9-a09621b3aa98,An attacker can uncover login names and passwo...,Enable FTPS or enforce the connection via the ...,NaN,NaN,Tries to login to a non FTPS enabled FTP servi...,NaN,NaN,NaN,NaN
1,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,c8e48745-ec7c-43ef-aaac-150f8c9c3392,An attacker can quickly break individual conne...,Disable the reported weak KEX algorithm(s)\n\n...,NaN,- 1024-bit MODP group / prime KEX algorithms:\...,Checks the supported KEX algorithms of the rem...,Product: cpe:/a:ietf:secure_shell_protocol\nMe...,NaN,NaN,NaN
2,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,c38e1e4c-c0bb-4af9-b23f-71d9e2e89c57,NaN,Disable the reported weak host key algorithm(s).,NaN,NaN,Checks the supported host key algorithms of th...,Product: cpe:/a:ietf:secure_shell_protocol\nMe...,NaN,NaN,NaN
3,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,c1374113-02e5-4f4d-a108-2031980f3d7b,NaN,Disable the reported weak encryption algorithm...,NaN,- The 'arcfour' cipher is the Arcfour stream c...,Checks the supported encryption algorithms (cl...,Product: cpe:/a:ietf:secure_shell_protocol\nMe...,NaN,NaN,NaN
4,21186,NaN,NaN,NaN,192.168.1.172,tcp,8009,AJP Connector Detection,There is an AJP connector listening on the rem...,The remote host is running an AJP (Apache JSer...,...,3e30d0a2-3168-493e-bcde-0571890ce8ab,NaN,Disable the reported weak MAC algorithm(s).,NaN,NaN,Checks the supported MAC algorithms (client-to...,Product: cpe:/a:ietf:secure_shell_protocol\nMe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2659,10407,NaN,2.6,Low,192.168.1.172,tcp,6000,X Server Detection,An X11 server is listening on the remote host,The remote host is running an X11 server. X11...,...,c88e1920-9975-4814-827c-266848a6c805,An attacker can uncover sensitive data by snif...,Run the session over an encrypted channel prov...,NaN,NaN,\nDetails:\nVNC Server Unencrypted Data Transm...,NaN,NaN,NaN,NaN
2660,10407,NaN,2.6,Low,192.168.1.172,tcp,6000,X Server Detection,An X11 server is listening on the remote host,The remote host is running an X11 server. X11...,...,69f801d9-8220-469b-b101-713938354363,An attacker could use this situation to compro...,Enforce the transmission of sensitive data via...,Hosts / applications which doesn't enforce the...,NaN,Evaluate previous collected information and ch...,NaN,NaN,NaN,NaN
2661,10407,NaN,2.6,Low,192.168.1.172,tcp,6000,X Server Detection,An X11 server is listening on the remote host,The remote host is running an X11 server. X11...,...,9691f7b9-bc97-42a9-81ae-668ba63d6594,"By default, Distributed Ruby does not impose r...",Administrators of environments that rely on Di...,NaN,NaN,Send a crafted command to the service and chec...,NaN,NaN,NaN,NaN
2662,10407,NaN,2.6,Low,192.168.1.172,tcp,6000,X Server Detection,An X11 server is listening on the remote host,The remote host is running an X11 server. X11...,...,c2940fa8-f8e7-4484-b68b-454522534b19,An EOL version of an OS is not receiving any s...,Upgrade the OS on the remote host to a version...,NaN,NaN,Checks if an EOL version of an OS is present o...,Product: cpe:/o:canonical:ubuntu_linux:8.04\nM...,NaN,NaN,NaN
